This notebook runs a tiny demo of a [GPTQd StableLM
3B](https://huggingface.co/ethzanalytics/stablelm-tuned-alpha-3b-gptq-4bit-128g) on a
classification task using CAPPr and then via sampling.

In [1]:
# check correct CUDA version
import torch

_cuda_version = torch.version.cuda
_msg = (
    "Change the pip install auto-gptq command to the one for "
    f"{_cuda_version} based on the list here: "
    "https://github.com/PanQiWei/AutoGPTQ#quick-installation"
)

assert _cuda_version == "11.8", _msg

I'm gonna install `cappr` from source b/c sometimes I use this notebook to statistically gut check code changes.

In [ ]:
!python -m pip install "cappr[demos] @ git+https://github.com/kddubey/cappr.git" \
git+https://github.com/huggingface/transformers \
auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ \
optimum

Uncomment and run this:

In [3]:
# !git lfs install
# !git clone https://huggingface.co/ethzanalytics/stablelm-tuned-alpha-3b-gptq-4bit-128g

In [4]:
!ls stablelm-tuned-alpha-3b-gptq-4bit-128g

config.json			  README.md
generation_config.json		  special_tokens_map.json
gptq_model-4bit-128g.safetensors  tokenizer_config.json
quantize_config.json		  tokenizer.json


In [5]:
from auto_gptq import AutoGPTQForCausalLM
import torch
from transformers import AutoTokenizer, GenerationConfig, pipeline

from cappr.huggingface import classify as fast
from cappr.huggingface import classify_no_cache as slow

In [6]:
_msg = (
    "This notebook must run on a GPU. A T4 instance is sufficient for the models "
    "tested here."
)
assert torch.cuda.is_available(), _msg

In [8]:
quantized_model_dir = "stablelm-tuned-alpha-3b-gptq-4bit-128g"
model = AutoGPTQForCausalLM.from_quantized(
    quantized_model_dir, use_triton=False, use_safetensors=True
)
tokenizer = AutoTokenizer.from_pretrained("StabilityAI/stablelm-tuned-alpha-7b")

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


In [9]:
# warm up model
_ = model(**tokenizer(["warm up"], return_tensors="pt").to(model.device))

Chat format is pulled from [the HF page](https://huggingface.co/stabilityai/stablelm-tuned-alpha-7b).

In [10]:
stablelm_chat_template = """
<|SYSTEM|># {system_prompt}
<|USER|>{user_message}<|ASSISTANT|>
""".strip("\n")

In [11]:
# Define a classification task
feedback_types = (
    "the product is too expensive",
    "the product uses low quality materials",
    "the product is difficult to use",
    "the product is great",
)


# Write a prompt
def prompt_func(product_review: str) -> str:
    system_prompt = "You are an expert at summarizing product reviews."
    user_message = f"This product review: {product_review}\n" "is best summarized as:"
    return stablelm_chat_template.format(
        system_prompt=system_prompt, user_message=user_message
    )


# Supply the texts you wanna classify
product_reviews = [
    "I can't figure out how to integrate it into my setup.",
    "Yeah it's pricey, but it's definitely worth it.",
]
prompts = [prompt_func(product_review) for product_review in product_reviews]
completions = feedback_types

In [12]:
print(prompts[0])

<|SYSTEM|># You are an expert at summarizing product reviews.
<|USER|>This product review: I can't figure out how to integrate it into my setup.
is best summarized as:<|ASSISTANT|>


In [13]:
pred_probs_fast = fast.predict_proba(
    prompts, completions, model_and_tokenizer=(model, tokenizer)
)

In [14]:
pred_probs_slow = slow.predict_proba(
    prompts, completions, model_and_tokenizer=(model, tokenizer)
)

In [15]:
pred_probs_fast.round(3)

array([[0.278, 0.026, 0.438, 0.258],
       [0.348, 0.073, 0.206, 0.373]])

In [16]:
pred_probs_slow.round(3)

array([[0.273, 0.024, 0.431, 0.271],
       [0.347, 0.072, 0.199, 0.383]])

Differences in the `fast` and `slow` module probabilities are not as small as I was
hoping.

In [17]:
product_reviews

["I can't figure out how to integrate it into my setup.",
 "Yeah it's pricey, but it's definitely worth it."]

In [18]:
feedback_types

('the product is too expensive',
 'the product uses low quality materials',
 'the product is difficult to use',
 'the product is great')

Both predictions are correct. In the first product review, the 3rd class' probability is
the highest. In the second product review, the 4th class' probability is highest
(followed closely by the first class). Pretty impressive for an aggressively quantized
3B parameter model!

TODO: study `end_of_prompt` more closely. Or consider using a non-chat model?

The baseline to beat is sampling from the LM:

In [19]:
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

generation_config = GenerationConfig(
    max_new_tokens=100,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    batch_size=1,
)

The model 'GPTNeoXGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartForCausalLM', 'Prop

Doubt

In [20]:
user_message = """
Every product review belongs to one of these lettered categories:
A. the product is too expensive
B. the product uses low quality materials
C. the product is difficult to use
D. the product is great

Which category does the following product review belong to?
Product review: "I can't figure out how to integrate it into my setup."

Respond only with the letter choice: A or B or C or D.
""".strip("\n")

system_prompt = (
    "You are an expert at categorizing product reviews. "
    "Respond with the letter corresponding to the category which the given "
    "product review belongs to."
)

prompt = stablelm_chat_template.format(
    system_prompt=system_prompt, user_message=user_message
)
print(prompt)

<|SYSTEM|># You are an expert at categorizing product reviews. Respond with the letter corresponding to the category which the given product review belongs to.
<|USER|>Every product review belongs to one of these lettered categories:
A. the product is too expensive
B. the product uses low quality materials
C. the product is difficult to use
D. the product is great

Which category does the following product review belong to?
Product review: "I can't figure out how to integrate it into my setup."

Respond only with the letter choice: A or B or C or D.<|ASSISTANT|>


In [21]:
sequences = generator(
    prompt,
    generation_config=generation_config,
    pad_token_id=generator.tokenizer.eos_token_id,  # suppress "Setting ..."
)
for seq in sequences:
    response = seq["generated_text"].removeprefix(prompt)
    print(response)

A. The product is too expensive.


Wrong. You can try setting `do_sample=True` in the `GenerationConfig` but it rarely
gives correct and consistently parseable results.